In [81]:
import matplotlib.pyplot as plt
from itertools import product

In [137]:
import numpy as np

def sinkhorn_with_polarization(C, a, b, P, P_max, epsilon, lambda_p, dual_lr=0.01, precision=1e-4, max_iter=1500):
    """
    Sinkhorn algorithm with polarization constraint.
    
    Args:
        C (ndarray): Cost matrix.
        a (ndarray): Marginal distribution for rows.
        b (ndarray): Marginal distribution for columns.
        P (ndarray): Polarization scores for columns.
        P_max (float): Maximum allowable polarization.
        epsilon (float): Entropic regularization parameter.
        lambda_p (float): Polarization penalty weight.
        dual_lr (float): Learning rate for dual variable (kappa).
        precision (float): Convergence threshold.
        max_iter (int): Maximum number of iterations.
        
    Returns:
        P_opt (ndarray): Optimal transport plan.
        polarization_history (list): Evolution of polarization over iterations.
        kappa (float): Final dual variable for polarization constraint.
    """
    n, m = C.shape
    K = np.exp(-C / epsilon)  # Initialize Gibbs kernel
    u = np.ones((n, 1))  # Row scaling factors
    v = np.ones((m, 1))  # Column scaling factors
    kappa = 0  # Dual variable for polarization constraint
    polarization_history = []

    for iteration in range(max_iter):
        # Update scaling factors
        u = a / np.maximum(K @ v, 1e-300)  # Avoid division by zero
        v = b / np.maximum(K.T @ u, 1e-300)
        
        # Compute transport plan
        P_opt = np.diagflat(u.flatten()) @ K @ np.diagflat(v.flatten())
        
        # Compute total polarization
        total_polarization = np.sum(P_opt * P[np.newaxis, :])
        polarization_history.append(total_polarization)
        
        # Update dual variable for polarization constraint
        if total_polarization > P_max:
            polarization_violation = total_polarization - P_max
            kappa += dual_lr * polarization_violation
        
        # Update cost matrix and kernel
        C_adjusted = C + lambda_p * P[np.newaxis, :] + kappa * P[np.newaxis, :]
        K = np.exp(-C_adjusted / epsilon)
        
         # Convergence check
        norm_diff = np.abs(np.sum(P_opt) - np.sum(a))
        if norm_diff < precision and abs(total_polarization - P_max) <=  0.1:
            print(f"Converged at iteration {iteration} with norm_diff = {norm_diff:.2e} and total_polarization = {total_polarization:.2e}.")
            break
    else:
        print(f"Reached maximum iterations ({max_iter}) with norm_diff = {norm_diff:.2e} and total_polarization = {total_polarization:.2e}.")

    return P_opt, polarization_history, kappa


In [138]:
def parameter_tuning(C, a, b, P, parameter_grid, precision=1e-1, max_iter=3000):
    """
    Tune parameters for the Sinkhorn algorithm with polarization.

    Args:
        C (ndarray): Cost matrix.
        a (ndarray): Marginal distribution for rows.
        b (ndarray): Marginal distribution for columns.
        P (ndarray): Polarization scores for columns.
        parameter_grid (dict): Parameter grid with lists of possible values.
        precision (float): Convergence threshold.
        max_iter (int): Maximum number of iterations.

    Returns:
        results (list): List of results for each parameter combination.
    """
    results = []
    param_combinations = list(product(
        parameter_grid['epsilon'],
        parameter_grid['lambda_p'],
        parameter_grid['dual_lr'],
        parameter_grid['P_max']
    ))

    for epsilon, lambda_p, dual_lr, P_max in param_combinations:
        print(f"Testing parameters: epsilon={epsilon}, lambda_p={lambda_p}, dual_lr={dual_lr}, P_max={P_max}")
        
        P_opt, polarization_history, kappa = sinkhorn_with_polarization(
            C, a, b, P, P_max, epsilon, lambda_p, dual_lr, precision, max_iter
        )
        
        # Log results
        norm_diff = np.abs(np.sum(P_opt) - np.sum(a))
        final_polarization = polarization_history[-1]
        converged = (norm_diff < precision)

        results.append({
            'epsilon': epsilon,
            'lambda_p': lambda_p,
            'dual_lr': dual_lr,
            'P_max': P_max,
            'iterations': len(polarization_history),
            'final_polarization': final_polarization,
            'norm_diff': norm_diff,
            'converged': converged
        })

        print(f"  Result: {'Converged' if converged else 'Did not converge'}, "
              f"Iterations: {len(polarization_history)}, "
              f"Final Polarization: {final_polarization:.4f}, "
              f"Norm Diff: {norm_diff:.4e}")

    return results


In [139]:
# Example setup
n_users = 100
n_content = 80
np.random.seed(42)

# Generate distributions and polarization scores
a = np.random.rand(n_users, 1)
a /= a.sum()
b = np.random.rand(n_content, 1)
b /= b.sum()
C = np.random.rand(n_users, n_content)  # Cost matrix
P = np.random.rand(n_content)  # Polarization scores

# Constraints and regularization
P_max = 0.1
epsilon = 0.05
lambda_p = 10.0
dual_lr = 0.5

# Define parameter grid
parameter_grid = {
    'epsilon': [0.05, 0.1, 0.2],
    'lambda_p': [10.0, 20.0, 50.0],
    'dual_lr': [0.01, 0.1, 0.5],
    'P_max': [0.1, 0.2]
}

# Run parameter tuning
results = parameter_tuning(C, a, b, P, parameter_grid)

Testing parameters: epsilon=0.05, lambda_p=10.0, dual_lr=0.01, P_max=0.1
Reached maximum iterations (3000) with norm_diff = 1.11e-16 and total_polarization = 4.39e-01.
  Result: Converged, Iterations: 3000, Final Polarization: 0.4391, Norm Diff: 1.1102e-16
Testing parameters: epsilon=0.05, lambda_p=10.0, dual_lr=0.01, P_max=0.2
Reached maximum iterations (3000) with norm_diff = 1.11e-16 and total_polarization = 4.39e-01.
  Result: Converged, Iterations: 3000, Final Polarization: 0.4391, Norm Diff: 1.1102e-16
Testing parameters: epsilon=0.05, lambda_p=10.0, dual_lr=0.1, P_max=0.1
Reached maximum iterations (3000) with norm_diff = 3.87e-01 and total_polarization = 1.50e-01.
  Result: Did not converge, Iterations: 3000, Final Polarization: 0.1502, Norm Diff: 3.8660e-01
Testing parameters: epsilon=0.05, lambda_p=10.0, dual_lr=0.1, P_max=0.2
Reached maximum iterations (3000) with norm_diff = 2.56e-01 and total_polarization = 2.29e-01.
  Result: Did not converge, Iterations: 3000, Final Pola

In [140]:
def select_best_parameters(results):
    """
    Select the best parameters based on polarization proximity to P_max among converged results.

    Args:
        results (list): List of results with metrics for each parameter combination.

    Returns:
        best_result (dict): Best parameter set based on the lowest polarization error.
    """
    # Filter only converged results
    converged_results = [x for x in results if x['converged']]
    
    if not converged_results:
        print("No parameter combination converged.")
        return None
    
    # Minimize polarization error among converged results
    best_result = min(converged_results, key=lambda x: abs(x['final_polarization'] - x['P_max']))
    return best_result

# Select the best parameters based on their respective P_max values
best_result = select_best_parameters(results)

if best_result:
    print("\nBest Parameters:")
    print(f"Epsilon: {best_result['epsilon']}")
    print(f"Lambda_p: {best_result['lambda_p']}")
    print(f"Dual Learning Rate: {best_result['dual_lr']}")
    print(f"P_max: {best_result['P_max']}")
    print(f"Iterations: {best_result['iterations']}")
    print(f"Final Polarization: {best_result['final_polarization']}")
    print(f"Norm Diff: {best_result['norm_diff']}")
    print(f"Converged: {best_result['converged']}")


Best Parameters:
Epsilon: 0.1
Lambda_p: 10.0
Dual Learning Rate: 0.1
P_max: 0.2
Iterations: 3000
Final Polarization: 0.3724488343628324
Norm Diff: 0.0701482473091588
Converged: True


In [80]:
# Run Sinkhorn algorithm with polarization constraint
P_opt, polarization_history, kappa = sinkhorn_with_polarization(
    C, a, b, P, P_max, epsilon, lambda_p, dual_lr
)

# Results
print("Optimal transport plan:")
print(P_opt)
print(f"Final total polarization: {polarization_history[-1]}")
print(f"Dual variable (kappa): {kappa}")
print("Polarization history:")
print(polarization_history)


Reached maximum iterations (5000) with norm_diff = 5.40e-01 and total_polarization = 9.95e-02.
Optimal transport plan:
[[1.82744530e-06 0.00000000e+00 0.00000000e+00 ... 1.76532084e-05
  6.18149209e-12 1.06840870e-11]
 [1.57659220e-04 0.00000000e+00 0.00000000e+00 ... 2.42996817e-05
  2.28084800e-14 6.25902074e-10]
 [9.32193127e-13 0.00000000e+00 0.00000000e+00 ... 8.59378861e-10
  4.78047487e-13 1.03934786e-08]
 ...
 [1.64590721e-11 0.00000000e+00 0.00000000e+00 ... 2.93485931e-12
  1.61395928e-14 7.37159996e-14]
 [6.76369363e-12 0.00000000e+00 0.00000000e+00 ... 5.88904040e-05
  4.33044954e-07 1.18397216e-05]
 [2.45412127e-09 0.00000000e+00 0.00000000e+00 ... 3.65331532e-09
  4.46971198e-14 4.16328804e-11]]
Final total polarization: 0.09954931486775594
Dual variable (kappa): 66.09573031207596
Polarization history:
[0.49613252242351386, 0.49613252242351374, 0.4961325224235138, 0.4961325224235138, 0.49613252242351386, 0.4961325224235138, 0.49613252242351386, 0.4961325224235139, 0.49613